In [1]:

from IPython.display import display

import re
from pathlib import Path

import pandas as pd

from src.processing.taxation_list import _open_file_taxation_list
from src.parsing.splitting import Splitter
from src.parsing.regular_templates import Templates
from src.parsing.tree_parsing import Parser

In [2]:
file_path = Path().absolute().parent / "data/example_1_taxation_list.xlsx"

is_import_first_row = False
column_mapping = {
    'number': 0,
    'name': 1,
    'quantity': 2,
    'height': 3,
    'diameter': 4,
    'quality': 5
}
table_data_df = _open_file_taxation_list(file_path, is_import_first_row, column_mapping)
table_data_df

,origin_number,name,quantity,height,diameter,quality
0,1,береза,14 м2,1,-,Хорошее
1,2,ольха,14 м2,"2,5","0,08",Хорошее
2,"3,4",Ольха серая,2,"5,5","4,5",Хорошее
3,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее
4,16,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее
5,7,береза,1,"2,5",-,Хорошее
6,8,береза,72 м2,"1,5",-,Хорошее
7,9,ольха,72 м2,2,"0,06",Хорошее
8,10,Яблоня домашняя,2 ствола,"1,5х2","0,04х2",Хорошее
9,11,Яблоня домашняя,4,1.5х4,0.04х4,Хорошее


In [3]:
def filter_quantity_1(df: pd.DataFrame) -> pd.Series:
    df_1 = df[df['quantity'].str.contains(r'^\d+$')]
    return df_1[df_1['quantity'].astype(int) > 1]

def filter_quantity_2(df: pd.DataFrame) -> pd.Series:
    return df[~df['quantity'].str.contains(r'^\d+$')]

def filter_quantity_3(df: pd.DataFrame) -> pd.Series:
    return df[df['quantity'].str.contains(r'^1$')]

display(table_data_df.pipe(filter_quantity_1))
display(table_data_df.pipe(filter_quantity_2))
display(table_data_df.pipe(filter_quantity_3))

,origin_number,name,quantity,height,diameter,quality
2,"3,4",Ольха серая,2,"5,5","4,5",Хорошее
9,11,Яблоня домашняя,4,1.5х4,0.04х4,Хорошее
10,12,Яблоня домашняя,3,"1,5х3","0,03х3",Хорошее
12,14,Яблоня домашняя,2,1.5х2,0.03х2,Хорошее
17,19а-в,Берёза пушистая,3,"0,4","0,01",Хорошее
19,"20,21-22, 23",Ольха серая,4,5х4,"5,4,8х2",Хорошее
20,24-25,Берёза пушистая,2,"0,4","0,01",Хорошее
21,26,Берёза пушистая,3,0.4,0.01,Хорошее
24,27.2-27.3,Берёза пушистая,2,"0,4","0,01",Хорошее


,origin_number,name,quantity,height,diameter,quality
0,1,береза,14 м2,1,-,Хорошее
1,2,ольха,14 м2,"2,5","0,08",Хорошее
3,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее
4,16,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее
6,8,береза,72 м2,"1,5",-,Хорошее
7,9,ольха,72 м2,2,"0,06",Хорошее
8,10,Яблоня домашняя,2 ствола,"1,5х2","0,04х2",Хорошее
11,13,Яблоня домашняя,2 ствола,"1,5х2","0,04х2",Хорошее; плохое
13,15,Алыча,19 м.п.,2х2,"0,07х2",Хорошее
14,16,Алыча,6 м.п.,3,"0,12",Хорошее


,origin_number,name,quantity,height,diameter,quality
5,7,береза,1,"2,5",-,Хорошее
16,18,Смородина чёрная,1,"0,5",-,Хорошее
18,19 г,Берёза пушистая,1,"0,4","0,01",Хорошее


In [3]:
def split(series: pd.Series) -> list[dict[str, str|int|float|None]]:
    split_row = []
    split_numbers = Splitter.number(series['origin_number'])
    specie = Parser.get_specie(series['name'])
    quantity = Parser.get_type_tree_object(series['quantity'])
    print(split_numbers, specie, quantity)
    # quantity_tree = int(series['quantity'].strip()) if not (('ствол' in series['quantity']) or ('м' in series['quantity'])) else 1
    # match_trunk = re.search(Templates.TRUNKS, series['quantity'])
    # quantity_trunk = int(match_trunk.group(1)) if match_trunk else 1
    # heights = Splitter.size(series['height'], quantity_trunk)
    # diameters = Splitter.size(series['diameter'], quantity_trunk)
    # join_items = lambda items: ','.join([str(item) for item in items])
    # row_dict = {
    #     'origin_number': series['origin_number'], 
    #     'split_number': join_items(split_numbers),
    #     'name': series['name'],
    #     'quantity': series['quantity'],
    #     'quantity_tree': quantity_tree,
    #     'height': series['height'],
    #     'heights': join_items(heights),
    #     'diameter': series['diameter'],
    #     'diameters': join_items(diameters),
    #     'quality': series['quality'],
    # }
    # split_row.append(row_dict)
    # return row_dict
    


# table_data_df[['quantity_tree', 'many_trunks']] = table_data_df[['origin_number', 'quantity']].transpose().apply(lambda x: get_quantity(x)).transpose()

In [4]:
for idx, row in table_data_df.iterrows():
    split(row)

['1'] Specie(breed_genus='береза', breed_specie='', shrub=False) <class 'src.objects.tree.TreeContour'>
['2'] Specie(breed_genus='ольха', breed_specie='', shrub=False) <class 'src.objects.tree.TreeContour'>
['3', '4'] Specie(breed_genus='Ольха', breed_specie='серая', shrub=False) <class 'src.objects.tree.Tree'>
['5'] Specie(breed_genus='береза', breed_specie='', shrub=False) <class 'src.objects.tree.Tree'>
['16'] Specie(breed_genus='Ива', breed_specie='ломкая', shrub=False) <class 'src.objects.tree.Tree'>
['7'] Specie(breed_genus='береза', breed_specie='', shrub=False) <class 'src.objects.tree.Tree'>
['8'] Specie(breed_genus='береза', breed_specie='', shrub=False) <class 'src.objects.tree.TreeContour'>
['9'] Specie(breed_genus='ольха', breed_specie='', shrub=False) <class 'src.objects.tree.TreeContour'>
['10'] Specie(breed_genus='Яблоня', breed_specie='домашняя', shrub=False) <class 'src.objects.tree.Tree'>
['11'] Specie(breed_genus='Яблоня', breed_specie='домашняя', shrub=False) <clas

In [15]:
re.match(r'^\d+\s*м\.?п\.?$', '19м.п.')

<re.Match object; span=(0, 6), match='19м.п.'>

In [15]:
# split_table_dta_df = pd.DataFrame(columns=['origin_id', *table_data_df.columns])
row_dict = {
    'origin_number': 0, 
    'split_number': 'splitted_numbers',
    'name': 'name',
    'quantity': 'quantity',
    'quantity_tree': str('quantity_tree'),
    'height': 'height',
    'heights': str('heights'),
    'diameter': 'diameter',
    'diameters': str('diameters'),
    'quality': 'quality',
}
split_table_data_df = pd.DataFrame(columns=[*row_dict.keys()])


for idx, row in table_data_df.iterrows():
    # print(row['name'])
    # display(split(row))
    split_table_data_df = pd.concat([split_table_data_df, pd.DataFrame(split(row), index=[idx])])

split_table_data_df

,origin_number,split_number,name,quantity,quantity_tree,height,heights,diameter,diameters,quality
0,1,1,береза,14 м2,1,1,1,-,-,Хорошее
1,2,2,ольха,14 м2,1,"2,5","2,5","0,08","0,8",Хорошее
2,"3,4","3,4",Ольха серая,2,2,"5,5","5,5","4,5","4,5",Хорошее
3,5,5,береза,6 стволов,1,"4,4,4,4,4,4","4,4,4,4,4,4","5,5,5,5,5,5","5,5,5,5,5,5",Хорошее
4,16,16,Ива ломкая,54 ствола,1,"5х14, 4х30, 3х8,2,1","5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4,...","7х3,4х11, 2х40","7,7,7,4,4,4,4,4,4,4,4,4,4,4,2,2,2,2,2,2,2,2,2,...",Хорошее
5,7,7,береза,1,1,"2,5","2,5",-,-,Хорошее
6,8,8,береза,72 м2,1,"1,5","1,5",-,-,Хорошее
7,9,9,ольха,72 м2,1,2,2,"0,06","0,6",Хорошее
8,10,10,Яблоня домашняя,2 ствола,1,"1,5х2","1,5,5","0,04х2","0,4,4",Хорошее
9,11,11,Яблоня домашняя,4,4,1.5х4,"1.5,1.5,1.5,1.5",0.04х4,"0.04,0.04,0.04,0.04",Хорошее
